In [2]:
## EXAMPLE ON PYBULLET

import pybullet as p
import time
import pybullet_data
import numpy as np

"""
physicsClient = p.connect(p.GUI)  # or p.DIRECT for non-graphical version
p.setAdditionalSearchPath(pybullet_data.getDataPath())  # optionally
p.setGravity(0, 0, -10)
planeId = p.loadURDF("plane.urdf")
startPos = [0, 0, 1]
startOrientation = p.getQuaternionFromEuler([0, 0, 0])
boxId = p.loadURDF("r2d2.urdf", startPos, startOrientation)
# set the center of mass frame (loadURDF sets base link frame)
for i in range(10000):
    p.stepSimulation()
    time.sleep(1.0 / 240.0)
cubePos, cubeOrn = p.getBasePositionAndOrientation(boxId)
print(cubePos, cubeOrn)
p.disconnect() """

## IMPLEMENTATION OF ACROBOT

#   SET THE LENGHT AND HEIGHT OF ACROBOT
lenghFirstLink = 1.0
lenghtSecondLink = 1.0

#   SET THE MASS OF THE FIRST LINK
massFirstLink = 1.0
#   SET THE MASS OF THE SECOND LINK
massSecondLink = 1.0
#   GRAVITY ACCELLERATION
g = 9.81


# DEFINING THE DYNAMIC OF THE SYSTEM INTO A FUNCTION
def dynamic(state, action):
    theta1, theta2, theta1dot, theta2dot = state
    tau = action

    c1 = np.cos(theta1)
    s1 = np.sin(theta1)
    c2 = np.cos(theta2)
    s2 = np.sin(theta2)
    c12 = np.cos(theta1 + theta2)
    s12 = np.sin(theta1 + theta2)

    #   MASS MATRIX
    massMatrix = np.zeros((2, 2))
    # ** is an elevation to 2
    massMatrix[0, 0] = (
        (massFirstLink + massSecondLink) * lenghFirstLink**2
        + massSecondLink * lenghtSecondLink**2
        + 2 * massSecondLink * lenghFirstLink * lenghtSecondLink * c2
    )
    massMatrix[0, 1] = (
        massSecondLink * lenghtSecondLink**2
        + massSecondLink * lenghFirstLink * lenghtSecondLink * c2
    )
    massMatrix[1, 0] = massMatrix[0, 1]
    massMatrix[1, 1] = massSecondLink * lenghtSecondLink**2

    # Coriolis and centrifugal forces
    coriolisForce = np.zeros((2, 2))
    coriolisForce[0, 0] = (
        -2 * massSecondLink * lenghFirstLink * lenghtSecondLink * s2 * theta2dot
    )
    coriolisForce[0, 1] = (
        -massSecondLink * lenghFirstLink * lenghtSecondLink * s2 * theta2dot
    )
    coriolisForce[1, 0] = (
        massSecondLink * lenghFirstLink * lenghtSecondLink * s2 * theta1dot
    )
    coriolisForce[1, 1] = 0

    # Gravity forces
    gravityForce = np.zeros((2, 1))
    gravityForce[0, 0] = (
        massFirstLink + massSecondLink
    ) * g * lenghFirstLink * s1 + massSecondLink * g * lenghtSecondLink * s12
    gravityForce[1, 0] = massSecondLink * g * lenghtSecondLink * s12

    # Compute accelerations
    # @ is an operator for matrix multiplication
    tau = np.clip(tau, -10, 10)  # Clip action to prevent large torque values
    accelleration = np.linalg.inv(massMatrix) @ (
        tau - coriolisForce @ np.array([[theta1dot], [theta2dot]]) - gravityForce
    )

    return np.array([theta1dot, theta2dot, accelleration[0, 0], accelleration[1, 0]])

    

# Control policy (swing-up and stabilization)
def control_policy(state, t):
    theta1, theta2, theta1_dot, theta2_dot = state

    # Swing-up control
    if abs(theta1) < np.pi / 2 and abs(theta2) < np.pi / 2:
        return -0.5 * theta1_dot - 1.5 * theta2_dot  # Swing-up torque

    # Stabilization control
    return (
        -10 * theta1 - 5 * theta1_dot - 5 * theta2 - 3 * theta2_dot
    )  
    # Stabilization torque


# Connect to the physics engine
physicsClient = p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0, 0, -g)

# -----------------------------------------------------------------
# Create the Acrobot in the simulation
# link1_radius = 0.1
# link2_radius = 0.1
# link1_height = lenghFirstLink
# link2_height = lenghtSecondLink

# # Create the floor
# planeId = p.loadURDF("plane.urdf")

# # Create the base of the Acrobot
# base_pos = [0, 0, 0]
# base_orient = p.getQuaternionFromEuler([0, 0, 0])
# baseId = p.loadURDF("acrobot_total.urdf", base_pos, base_orient)

# -----------------------------------------------------------------

# LOAD ACROBOT
acrobot_id = p.loadURDF("acrobot_total.urdf")

# Enable motors for the joints
p.setJointMotorControl2(acrobot_id, 0, p.POSITION_CONTROL, targetPosition=0, force=0)
p.setJointMotorControl2(acrobot_id, 1, p.VELOCITY_CONTROL, force=0)


# Initial state
theta1_0 = np.random.uniform(low=-np.pi, high=np.pi)
theta2_0 = np.random.uniform(low=-np.pi, high=np.pi)
theta1_dot_0 = np.random.uniform(low=-1.0, high=1.0)
theta2_dot_0 = np.random.uniform(low=-1.0, high=1.0)
state = np.array([theta1_0, theta2_0, theta1_dot_0, theta2_dot_0])

# Simulation parameters
dt = 0.01  # Time step
total_time = 10.0   # Total simulation time

# # SIMULATION STEP
# for _ in range(int(total_time / dt)):
#     # Get the current joint angles and velocities
#     joint_states = p.getJointStates(acrobot_id, [0, 1])
#     print(joint_states)
#     theta1, theta2 = [state[0] for state in joint_states]
#     theta1_dot, theta2_dot = [state[1] for state in joint_states]
    
#     # Apply the control policy to get the torque
#     action = control_policy([theta1, theta2, theta1_dot, theta2_dot], _ * dt)
    
#     # Apply the torque to the joints
#     p.setJointMotorControl2(acrobot_id, 1, p.TORQUE_CONTROL, force=action)
    
#     # Step the simulation
#     p.stepSimulation()
#     time.sleep(dt)

# Disconnect from the physics engine
p.disconnect()

((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0

((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0

((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0

((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0

((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (3.1970549691734282, 76.2713380354433, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (3.1970549691734282, -1.4210854715202004e-14, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (3.197055497056515, 0.00012669194074677037, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (3.197055497056515, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (3.1970559608821736, 0.00011131815814491297, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (3.197055373360453, -0.00014100521290916213, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (3.1970548856088534, -0.00011706038385694839, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (3.1970548856088534, -2.220446049250313e-16,

...

((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (3.045884476250533, -8.83786653712155, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (3.0120730956707162, -8.114731339156025, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.9826082845223363, -7.071554675611138, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.957956635217597, -5.916395833137462, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.938714294289253, -4.61816182280252, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.9255286644556024, -3.1645511600761425, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.9189055064404634, -1.5895579236333277, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.9190991730609492, 0.04647998891662297, (0.0, 0.

((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.6461528155700433, 0.09292011720439652, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.646498983232882, 0.08308023908124484, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.6448527334063967, -0.3950999583564361, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.643573082586756, -0.3071161967137872, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.6424463272986567, -0.2704212691438046, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.6407595215255473, -0.4048333855462185, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.639340584148846, -0.34054497040833764, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.6380317541427827, -0.31411920145521255,

((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.7082280125047276, -1.0712631413488851, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.7037170865894966, -1.082622219655434, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.699191198823369, -1.0862130638705931, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.694684026440115, -1.081721371980953, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.6902293652015636, -1.069118697252383, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.685861068522354, -1.048391203010258, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.681611063826574, -1.0200011269872395, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.6775111549542383, -0.9839781293605337, (0.0, 

((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.6305606013837624, -0.37700533449673, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.628911820447034, -0.39570742481482346, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.6271797084495065, -0.41570687940656226, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.6253591058526364, -0.43694462324878025, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.623445166686672, -0.45934539983155853, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.621433655936228, -0.4827625801064662, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.6193207853794314, -0.5070889336312174, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.6171036491854998, -0.5321126865435801,

((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.41488528079883, 0.2103657534121971, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.4149537506701186, 0.016432769109249512, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.4144234033993794, -0.1272833449774552, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.4160079024782015, 0.38027977891734877, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.4166854808205858, 0.16261880217217273, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.4167084183529206, 0.005505007760354119, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.4162652806053733, -0.10635305941136708, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (2.415495222549919, -0.18481393330906248